In [1]:
import os, random
import numpy as np
import pandas as pd
import datetime
from pprint import pprint
from pathlib import Path
import configparser
import ezomero
from ezomero import rois
#   https://thejacksonlaboratory.github.io/ezomero/ezomero.html#module-ezomero
#   https://docs.openmicroscopy.org/ome-model/5.6.3/developers/roi.html
#   https://github.com/HdBraanker/Spectral_Flow_Workflow/blob/main/script.Rmd
home = str(Path.home())

datestr = datetime.datetime.now().strftime('%Y_%m_%d')
print(datestr)

2024_07_15


In [2]:
### Subset Quant File from QuPath, which includes class annotations
clsPts = pd.read_csv(r'{}\Downloads\FF-19-8372_B3_potentialClasses.tsv'.format(home), sep="\t")  
clsPts

,CellTypes,Centroid X µm,Centroid Y µm
0,Macrophage,7392.60,6118.90
1,Macrophage,8891.20,8592.00
2,Macrophage,10239.70,3308.50
3,Macrophage,6681.80,4956.40
4,Macrophage,7264.10,7724.60
...,...,...,...
895,NK,352.96,6628.80
896,NK,9703.60,7454.50
897,NK,1428.20,5365.70
898,NK,9262.30,682.16


In [3]:
# Settings for OMERO
config = configparser.RawConfigParser()
config.read(r'{}\Documents\myKeyVariables.cfg'.format(home))   
envVars = dict(config.items('OMERO'))

conn = ezomero.connect(envVars['user'], envVars['pass'], 'Nguyen.J_Lab', host=envVars['url'], port=4064, secure=True)
ezomero.print_projects(conn)

Projects:
	PCF_2024_Transplant_Reaction_in_LN:	1254


In [4]:
ezomero.print_datasets(conn, project=1254)
setDs = 2015

getImageByName = {}
imgs = ezomero.get_image_ids(conn, dataset=setDs)
for im in imgs:
    imgDesc, pix = ezomero.get_image(conn, im, xyzct=False, no_pixels=True)
    imName = imgDesc.getName().replace(".ome.tiff","")
    getImageByName[imName] = im

Datasets in Project "PCF_2024_Transplant_Reaction_in_LN":
	cbdLN-Post:	2016
	HDLN-Post:	2015
	HDLN-Pre:	2013
	iliacLN-Post:	2012
	-removed-:	2014


In [5]:
def generate_rgba_values(length=3):
    rgba_list = []
    for _ in range(length):
        rgba = (
            random.randint(0, 255),  # Red
            random.randint(0, 255),  # Green
            random.randint(0, 255),  # Blue
            # round(random.uniform(0, 1), 2)  # Alpha (0.0 to 1.0)
            0.96
        )
        rgba_list.append(rgba)
    return rgba_list

In [6]:
im = getImageByName['Slide1_B3.ome.tif']
myColors = generate_rgba_values()

## QuPath Exports in um, but OMERO uploads in px. Need pixel resolution.
resolution_um_per_pixel = 0.51

i=0
for cl in clsPts['CellTypes'].unique():
    subTbl = clsPts[clsPts["CellTypes"] == cl]
    print("{} - {} rows".format(cl,subTbl.shape[0]))
    thesePoints = []
    for ele in subTbl.iterrows():
        row_dict = ele[1].to_dict()
        ptObj = ezomero.rois.Point( x=(row_dict['Centroid X µm']/resolution_um_per_pixel), 
                                    y=(row_dict['Centroid Y µm']/resolution_um_per_pixel), 
                                    label=row_dict['CellTypes']
                                  )
        thesePoints.append( ptObj )
    ezomero.post_roi(conn, im, thesePoints, stroke_color=myColors[i])
    i+=1


Macrophage - 300 rows
B cell - 300 rows
NK - 300 rows
